# Setup

In [13]:
import os
import requests
import zipfile
from io import BytesIO
import pandas as pd
import re

In [14]:
def extract_pdf_file(url, path, name):
    '''
    Downloads a ZIP file from a given URL, extracts a PDF file starting with a specified prefix,
    and saves it to a specified directory with a new name.

    Args:
        url (str): The URL from which to download the ZIP file.
        prefix (str): The prefix to filter PDF files within the ZIP.
        path (str): The local directory path to save the extracted PDF file.
        name (str): The new name for the extracted PDF file, without the '.pdf' extension.

    Returns:
        str or None: The path to the saved PDF file if a PDF file with the specified prefix is found
                     and successfully saved; otherwise, None if no PDF file matches the criteria.
   
    '''
    # Download data
    response = requests.get(url, stream=True)

    # Open Zip
    with zipfile.ZipFile(BytesIO(response.content)) as zip_file:

        pdf_count = 0

        # Iterate over all files in Zip
        for file_name in zip_file.namelist():
            # Only process pdf files
            if file_name.endswith('.pdf'):

                # Write pdf to disk
                with zip_file.open(file_name) as file_content:
                    content = file_content.read()

                    # The name of the pdf is the row id plus the pdf_count plus the original file name
                    full_path = re.sub(r'[<>:"/\\|?*#;\r\n, -]', '_', file_name)

                    if len(full_path) > 104:
                        full_path = full_path[0:100] + '.pdf'
                        
                    full_path = f'{path}/{name}_{pdf_count}_{full_path}'
                    

                    with open(full_path, 'wb') as new_file:
                        new_file.write(content)

                    pdf_count += 1
    # Return None if no matchin file is found
    return None

# Parameters

In [15]:
# Link to the SG. OGD repository
SG_OGD_LINK = 'https://daten.stadt.sg.ch/api/explore/v2.1/catalog/datasets/traktandierte-geschaefte-sitzungen-stadtparlament-stgallen/exports/parquet?lang=de&timezone=Europe%2FZurich'

# Paths
DATA_DIRECTORY = 'data'
DOWNLOAD_DIRECTORY = f'{DATA_DIRECTORY}/pdfs'

In [16]:
# Crate folders if the don't exist
if not os.path.exists(DATA_DIRECTORY):
    os.makedirs(DATA_DIRECTORY)

if not os.path.exists(DOWNLOAD_DIRECTORY):
    os.makedirs(DOWNLOAD_DIRECTORY)

# Download

In [17]:
# Download source file from SG_OGD_LINK
df = pd.read_parquet(SG_OGD_LINK)

# Create id column from the date and issue number
df['id'] = df['sitzungsdatum'] + '-' + df['traktandennummer']

# Filter source data and only keep Interpellationen
df = df[df['ebene4'] == 'Stadtparlament: Parlamentarische Vorstösse']

In [18]:
# Download pdfs for all entries

file_path = [] # Placeholder to track issues where no file was found

for _, row in df.iterrows():
    extract_pdf_file(row['download_traktandum'], DOWNLOAD_DIRECTORY, row['id'])

In [19]:
downloaded_files = os.listdir(DOWNLOAD_DIRECTORY)

df_documents = []

for file_name in downloaded_files:
    file_elements = file_name.split('_')
    df_documents.append([file_elements[0], file_elements[1], file_name])

df_documents = pd.DataFrame(df_documents, columns=['id', 'document_no', 'file_name'])

df_merged = pd.merge(df, df_documents, how='left', on=['id'])

In [20]:
# Save source table and indicator if files were found to disk
df_merged.to_csv(f'{DATA_DIRECTORY}/source_table.csv', index=False, sep=';')